In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

In [2]:
nlp = spacy.load('en_core_web_lg')
reviews_df = pd.read_csv("amazon_product_reviews.csv", low_memory=False)
reviews_df.sample(10)

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
14929,AVpjEN4jLJeJML43rpUe,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,B018Y225IA,Amazon,"Computers/Tablets & Networking,Tablets & eBook...","841667103143,0841667103143,brandnewamazonkindl...",Amazon,2016-11-06T00:00:00.000Z,NaN,"2017-08-27T00:00:00Z,2017-08-09T00:00:00Z,2017...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5025500/review...,No wife loves reading books on it. It's a good...,Nice,NaN,NaN,Jay021
4554,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2017-01-23T00:00:00.000Z,2017-05-21T03:50:34Z,"2017-04-30T00:25:00.000Z,2017-06-07T08:14:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5025800/review...,It is a nice convenient tablet for carrying ar...,It is nice,NaN,NaN,Saundra
29678,AV1YE_muvKc47QAVgpwE,NaN,B00U3FPN4U,Amazon Fire Tv,"Back To College,College Electronics,College Tv...","848719057492,amazonfiretv/51454342,amazonfiret...",Amazon,2016-05-19T00:00:00.000Z,2017-09-20T05:35:59Z,"2017-08-25T22:41:05.219Z,2017-08-19T09:25:44.1...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/4370400/review...,I love that this guy supports 4K content since...,Great streaming device!!,NaN,NaN,CarlosCE
22604,AVpfl8cLLJeJML43AE3S,"Echo (White),,,\r\nEcho (White),,,","B00L9EPT8O,B01E6AO69U",Amazon,"Stereos,Remote Controls,Amazon Echo,Audio Dock...","echowhite/263039693056,echowhite/152558276095,...",Amazon,2017-07-07T00:00:00.000Z,NaN,"2017-09-28T00:00:00Z,2017-09-08T00:00:00Z,2017...",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5588528/review...,Decided to add another Echo to the household. ...,My 2nd Echo,NaN,NaN,Dana
28903,AVpidLjVilAPnD_xEVpI,NaN,B0189XYY0Q,Amazon Fire,"Electronics,Tablets & E-Readers,Tablets,Back T...","841667101743,amazonfire/51441641,amazonfirehd1...",Amazon,2017-08-04T00:00:00.000Z,NaN,"2017-09-03T08:45:17.350Z,2017-08-27T11:00:57.0...",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5386601/review...,This is a great tablet for the money. The only...,Great for the price,NaN,NaN,NJGiants
30412,AV1YE_muvKc47QAVgpwE,NaN,B00U3FPN4U,Amazon Fire Tv,"Back To College,College Electronics,College Tv...","848719057492,amazonfiretv/51454342,amazonfiret...",Amazon,2016-12-04T00:00:00.000Z,2017-09-20T05:35:56Z,"2017-08-25T22:22:17.937Z,2017-08-19T09:27:03.8...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/4370400/review...,Great product! Lots of channels from everywher...,Awesome!,NaN,NaN,Lolalola
4190,AVphgVaX1cnluZ0-DR74,"Fire Tablet, 7 Display, Wi-Fi, 8 GB - Includes...",B018Y229OU,Amazon,"Fire Tablets,Tablets,Computers & Tablets,All T...",firetablet7displaywifi8gbincludesspecialoffers...,Amazon,2016-01-25T00:00:00.000Z,2017-07-02T13:15:46Z,"2017-06-07T08:13:00.000Z,2017-04-30T00:17:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5025800/review...,"Haven't used it much, but so far I like it! Mo...","Haven't used it much, but so far I like it!",NaN,NaN,Momma
30838,AV1YE_muvKc47QAVgpwE,NaN,B00U3FPN4U,Amazon Fire Tv,"Back To College,College Electronics,College Tv...","848719057492,amazonfiretv/51454342,amazonfiret...",Amazon,2017-01-16T00:00:00.000Z,2017-09-20T05:35:55Z,"2017-08-25T22:21:39.726Z,2017-08-19T09:26:44.3...",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/4370400/review...,Awesome alternative to cable television. Being...,Fire TV,NaN,NaN,John
32645,AV1YE_muvKc47QAVgpwE,NaN,B00U3FPN4U,Amazon Fire Tv,"Back To College,College Electronics,College Tv...","848719057492,amazonfiretv/51454342,amazonfiret...",Amazon,2017-05-15T00:00:00.000Z,2017-09-20T05:35:53Z,"2017-08-25T22:06:31.848Z,2017-08-19T09:35:50.0...",...,True,NaN,0

In [3]:
reviews_df = reviews_df[["reviews.text", "reviews.rating"]]
reviews_df.head(10)

,reviews.text,reviews.rating
0,This product so far has not disappointed. My c...,5.0
1,great for beginner or experienced person. Boug...,5.0
2,Inexpensive tablet for him to use and learn on...,5.0
3,I've had my Fire HD 8 two weeks now and I love...,4.0
4,I bought this for my grand daughter when she c...,5.0
5,This amazon fire 8 inch tablet is the perfect ...,5.0
6,"Great for e-reading on the go, nice and light ...",4.0
7,"I gave this as a Christmas gift to my inlaws, ...",5.0
8,Great as a device to read books. I like that i...,5.0
9,I love ordering books and reading them with th...,5.0


In [4]:
def label_reviews(rating):
    if rating == 5 or rating == 4:
        return "positive"
    elif rating == 2 or rating == 1:
        return "negative"
    else:
        return "neutral"

In [5]:
reviews_df["category"] = reviews_df["reviews.rating"].apply(label_reviews)
reviews_df["category"].value_counts()

category
positive    32316
neutral      1532
negative      812
Name: count, dtype: int64

In [6]:
reviews_df.head()

,reviews.text,reviews.rating,category
0,This product so far has not disappointed. My c...,5.0,positive
1,great for beginner or experienced person. Boug...,5.0,positive
2,Inexpensive tablet for him to use and learn on...,5.0,positive
3,I've had my Fire HD 8 two weeks now and I love...,4.0,positive
4,I bought this for my grand daughter when she c...,5.0,positive


In [8]:
label_encoder = LabelEncoder()
reviews_df["category"] = label_encoder.fit_transform(reviews_df["category"])

reviews_df["category"].value_counts()

category
2    32316
1     1532
0      812
Name: count, dtype: int64

In [10]:
max_words = 1000
max_len = 50
reviews = reviews_df["reviews.text"].values
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
X = pad_sequences(sequences, maxlen=max_len)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
# Sample reviews and their corresponding sentiment labels
reviews = [
    "The movie was great, I loved it!",
    "The movie was terrible, it was a waste of time.",
    "The acting was okay, but the plot was weak.",
    "I'm not sure how I feel about this movie.",
    "The movie exceeded my expectations."
]
sentiments = ['positive', 'negative', 'negative', 'neutral', 'positive']

# Tokenization and sequence padding
max_words = 1000
max_len = 50
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(reviews)
sequences = tokenizer.texts_to_sequences(reviews)
X = pad_sequences(sequences, maxlen=max_len)

# Convert labels to one-hot encoded vectors
label_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
y = np.array([label_dict[sentiment] for sentiment in sentiments])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=50, input_length=max_len))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer with 3 units for 3 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

# Make predictions
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Convert predicted labels back to original sentiments
inv_label_dict = {v: k for k, v in label_dict.items()}
y_pred_sentiments = [inv_label_dict[pred] for pred in y_pred]

# Calculate accuracy
accuracy = accuracy_score(sentiments, y_pred_sentiments)
print("Accuracy:", accuracy)